# tests with the MGRS ID to set geographic identifiers for 10m WEED reference data
## MGRS_ID10; S2_tile; EPSG; c_lat; c_lon; geometry

In [10]:
import geopandas as gpd
import pandas as pd
#import mgrs
import os

In [5]:
# init MGRS object
m = mgrs.MGRS()

In [16]:
# load the SK data for test
sk_path = os.path.normpath(r'C:\Users\BUCHHORM\Downloads\SK_v5_reference-points_EUNIS2012.csv')
df_sk = pd.read_csv(sk_path)

In [18]:
df_sk.head()

,fid,entry,x,y,valid_L1,valid_L2,valid_L3,lat,lon,label_L1,label_L2,label_L3
0,1,P5000001,4.849642e+06,2.811280e+06,True,True,True,17.118680,48.185489,C,C3,C3.4
1,2,P5000002,4.851750e+06,2.815627e+06,True,True,True,17.152425,48.222568,C,C3,C3.4
2,3,P5000003,4.854840e+06,2.816960e+06,True,True,True,17.195542,48.231791,C,C3,C3.4
3,4,P5000004,4.854535e+06,2.819285e+06,True,True,True,17.194425,48.252869,C,C3,C3.4
4,5,P5000005,4.855180e+06,2.819730e+06,True,True,True,17.203641,48.256284,C,C3,C3.4


In [30]:
# test with first entry
test = (*df_sk[['lat','lon']].iloc[0],)
test

(17.1186803, 48.1854891)

In [36]:
# convert latLon to MGRS_ID10 direct
m.toMGRS(*test, MGRSPrecision=4)

'39QTU00509485'

In [50]:
# convert to UTM via MGRS object  (zone, hemi, easting, northing)
m.MGRSToUTM(m.toMGRS(*test, MGRSPrecision=5))

(39, 'N', 200503.0, 1894850.0)

In [60]:
# convert to UTM via Marcel's tool  (long = X, lat = Y)
from eo_processing.utils.mgrs import LL_2_UTM, LL_2_MGRSid, UTM_2_LL
LL_2_UTM(test[1], test[0])

(200503.28893616836, 1894850.6240331712, 39, 'Q')

In [58]:
LL_2_MGRSid(test[1], test[0])

'39QTU'

In [62]:
def get_MGRSid10_center(longitude, latitude):
    """
    :param longitude: Longitude of the location in decimal degrees.
    :param latitude: Latitude of the location in decimal degrees.
    :return: The center coordinates in latitude and longitude of the nearest MGRS 10-meter grid, after rounding the UTM coordinates to the nearest 5 meters.
    """
    def round_to_nearest_5(value):
        """
        :param value: The value to be rounded.
        :return: The value rounded to the nearest 5.
        """
        return (int(value / 10.0) * 10) + 5

    easting, northing, zone, band = LL_2_UTM(longitude, latitude)
    rounded_easting = round_to_nearest_5(easting)
    rounded_northing = round_to_nearest_5(northing)
    return UTM_2_LL(rounded_easting, rounded_northing, zone, band)

In [66]:
c_center = get_MGRSid10_center(test[1],test[0])
print(c_center)

(48.18550457096998, 17.118720031874876)


In [68]:
m.toMGRS(c_center[1], c_center[0])

'39QTU0050594854'

In [108]:
# load global test set
globe_path = os.path.normpath(r'C:\Users\BUCHHORM\Downloads\latlon_global_test_set.csv')
df = pd.read_csv(globe_path)

In [110]:
df.head()

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [118]:
def process_row(row: pd.Series) -> None:
    """
    :param row: A pandas Series object containing 'longitude' and 'latitude'.
    :return: None. Prints out the MGRS id, formatted easting and northing coordinates, and MGRS result, or an error message if processing fails.
    """
    longitude, latitude = row.longitude, row.latitude
    try:
        mgrs_id = LL_2_MGRSid(longitude, latitude)
        easting, northing, zone, band = LL_2_UTM(longitude, latitude)
    except Exception as e:
        print(f"Error processing row: {row}", e)
        return
    formatted_easting = str(int(easting))[-5:-1]
    formatted_northing = str(int(northing))[-5:-1]
    mgrs_result = m.toMGRS(latitude, longitude, MGRSPrecision=4)
    print(f'{mgrs_id}{formatted_easting}{formatted_northing}   -  {mgrs_result}')

for row in df.itertuples():
    process_row(row)

31TCH85171137   -  31TCH85171137
39QYF91019334   -  39QYF91019334
42SUC80775615   -  42SUC80775615
20QPD28078667   -  20QPD28078667
20QMF92741458   -  20QMF92741458
34TDL30215611   -  34TDL30215611
38TNK03253542   -  38TNK03253542
19PDP93465155   -  19PDP93465155
33LZH13866007   -  33LZH13866007
30CWB83194633   -  30CWB83194633
20HMC46164783   -  20HMC46164783
02LNK93602213   -  02LNK93602213
33TVN66126263   -  33TVN66126263
53JLN76660410   -  53JLN76660410
19PCP94788437   -  19PCP94788437
38TQK19514682   -  38TQK19514682
33TYJ15096601   -  33TYJ15096601
21PTQ24345997   -  21PTQ24345997
46QBM30392185   -  46QBM30392185
31UFR04239568   -  31UFR04239568
30PXU56455332   -  30PXU56455332
35TLH76053237   -  35TLH76053237
39RVJ63726802   -  39RVJ63726802
35MRS24382668   -  35MRS24382668
31PDL24862894   -  31PDL24862894
20SLA34577741   -  20SLA34577741
50NKL47860169   -  50NKL47860169
20KMG37109887   -  20KMG37109887
22LDK00172608   -  22LDK00172608
18RTN58217088   -  18RTN58217088
46RBR46494

some tests with the new grid20id for the openEO 20x20km global processing grid. The grid20id functions should assign to each reference point the corresponding 20x20km grid cell so that a faster grouping for openEO processing purposes is possible.

In [1]:
from eo_processing.utils.mgrs import UTM_2_MGRSid, UTM_2_grid20id, LL_2_UTM, LL_2_grid20id, LL_2_MGRSid

In [16]:
# load the shapefile of the global terrestrial 100k grid
import eo_processing.resources
import importlib.resources as importlib_resources
import geopandas as gpd
import os
#grid = importlib_resources.files(eo_processing.resources).joinpath('global_terrestrial_MGRS100k_grid.gpkg')
grid = os.path.normpath(r'C:\Users\BUCHHORM\Downloads\world_mgrs\mgrs_region.shp')
gdf = gpd.read_file(os.path.normpath(grid))

In [17]:
gdf.head()

,GRID1MIL,GRID100K,LONGITUDE,LATITUDE,geometry
0,33L,SC,12.102867,-15.678288,"POLYGON ((12 -16, 12 -15.90909, 12 -15.81818, ..."
1,33L,TC,12.666855,-15.679626,"POLYGON ((13.13098 -16, 13.125 -16, 13.03125 -..."
2,33L,UC,13.599601,-15.684492,"POLYGON ((14.06538 -16, 14.0625 -16, 13.96875 ..."
3,33L,VC,14.532691,-15.687415,"POLYGON ((15 -16, 14.90625 -16, 14.8125 -16, 1..."
4,33L,WC,15.467309,-15.687415,"POLYGON ((15.93462 -16, 15.84375 -16, 15.75 -1..."


In [18]:
def get_grid20id_center(geometry):
    c_center = geometry.centroid
    return LL_2_grid20id(c_center.x, c_center.y)[:5]

In [19]:
gdf['grid100id'] = gdf.apply(lambda row: get_grid20id_center(row['geometry']), axis=1, result_type='expand')

In [20]:
gdf.head()

,GRID1MIL,GRID100K,LONGITUDE,LATITUDE,geometry,grid100id
0,33L,SC,12.102867,-15.678288,"POLYGON ((12 -16, 12 -15.90909, 12 -15.81818, ...",33εSC
1,33L,TC,12.666855,-15.679626,"POLYGON ((13.13098 -16, 13.125 -16, 13.03125 -...",33εTC
2,33L,UC,13.599601,-15.684492,"POLYGON ((14.06538 -16, 14.0625 -16, 13.96875 ...",33εUC
3,33L,VC,14.532691,-15.687415,"POLYGON ((15 -16, 14.90625 -16, 14.8125 -16, 1...",33εVC
4,33L,WC,15.467309,-15.687415,"POLYGON ((15.93462 -16, 15.84375 -16, 15.75 -1...",33εWC


In [21]:
# save to disk
out_path = os.path.normpath(r'C:\Users\BUCHHORM\Downloads\global_terrestrial_UTM100k_grid_unfiltered.gpkg')
gdf.to_file(out_path, driver='GPKG')

In [2]:
# test regarding leading zeros
zero = (0,0)
LL_2_UTM(*zero)

(166021.44308054057, 0.0, 31, 'N')

In [3]:
LL_2_grid20id(*zero)

'31λAA30'

In [4]:
LL_2_MGRSid(*zero)

'31NAA'

In [5]:
from eo_processing.utils.mgrs import LL_2_grid100id, LL_2_MGRSid10

In [6]:
LL_2_grid100id(*zero)

'31λAA'

In [7]:
LL_2_MGRSid10(*zero)

'31NAA66020000'